In [1]:
import os
import sys
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from skimage import io
import cv2

In [2]:
def calculate_distance(com1, com2):
    return (np.linalg.norm(com1 - com2))

In [3]:
csvpath = '/net/birdstore/Active_Atlas_Data/data_root/atlas_data/atlasV8/csv'
distancepath = '/net/birdstore/Active_Atlas_Data/data_root/atlas_data/atlasV8/distance'
atlaspath = '/net/birdstore/Active_Atlas_Data/data_root/brains_info/registration'

In [12]:
allen_path = os.path.join(atlaspath, 'Allen_25um_coronal.tif')
allen_img = io.imread(allen_path)
allen_img.dtype, allen_img.shape

(dtype('uint16'), (528, 320, 456))

In [ ]:
inpath = os.path.join(atlaspath, 'MD589_25um_sagittal.tif')
MD589_img = io.imread(inpath)
MD589_img.dtype, MD589_img.shape

In [13]:
sagittal = np.swapaxes(allen_img,0,2)
print(f'sagittal shape={sagittal.shape} dtype={sagittal.dtype}')
savepath = os.path.join(atlaspath, 'Allen_25um_sagittal.tif')
io.imsave(savepath, sagittal)

sagittal shape=(456, 320, 528) dtype=uint16


/tmp/ipykernel_1237166/2420175636.py:7: UserWarning: /net/birdstore/Active_Atlas_Data/data_root/brains_info/registration/Allen_25um_sagittal.tif is a low contrast image
  io.imsave(savepath, sagittal)


In [ ]:
padded = pad_volume(allen_img, MD589_img.shape)
savepath = os.path.join(atlaspath, 'Allen_25um_sagittal.tif')
io.imsave(savepath, padded)
padded.shape

In [ ]:
len(df_midbrain_allen)

In [ ]:
distances = []
sortme = {}

for (i1,row1), (i2,row2) in zip(df_midbrain_allen.iterrows(), df_midbrain_atlas.iterrows() ): 
    structure = row1['Structure']

    fig = plt.figure()
    plt.rcParams["figure.figsize"] = [12, 8]
    x1 = row1['X']
    y1 = row1['Y']
    z1 = row1['Z']
    x2 = row2['X']
    y2 = row2['Y']
    z2 = row2['Z']
    section = int(round(z1/25, 2))
    xdistance = abs(x1 - x2)
    ydistance = abs(y1 - y2)
    zdistance = abs(z1 - z2)
    distance = calculate_distance(np.array([x1,y1,z1]), np.array([x2,y2,z2]))
    slice = sagittal[:,:,section]
    title = f'{structure} at section={section}\n'
    title += f'x distance={round(xdistance,2)} um\n'
    title += f'y distance={round(ydistance,2)} um\n'
    title += f'z distance={round(zdistance,2)} um\n'
    title += f'  distance={round(distance,2)} um'
    plt.title(title)
    ax = plt.gca()
    ax.set_xlim([300, 600])
    ax.set_ylim(300,0)
    plt.scatter(x1/25, y1/25, s=40, marker='^', c='b', label=str('Allen ' + structure))
    plt.scatter(x2/25, y2/25, s=40, marker='o', c='g', label=str('Atlas ' + structure))
    plt.imshow(slice, cmap='gray')
    plt.legend()
    plt.close()
    outpath = os.path.join(distancepath, 'eps', f'{structure}.eps')
    fig.savefig(outpath, bbox_inches="tight", format='eps')
    outpath = os.path.join(distancepath, f'{structure}.png')
    fig.savefig(outpath, bbox_inches="tight")
    
    distances.append(distance)
    sortme[structure] = distance


In [ ]:
width = 0.25  # the width of the bars
multiplier = 0
x = np.arange(1) 
x + width

In [ ]:
errors = ['X error', 'Y error', 'Z error', 'Distance error']
x = np.arange(len(errors))  # the label locations
width = 0.05  # the width of the bars
for (i1,row1), (i2,row2) in zip(df_midbrain_allen.iterrows(), df_midbrain_atlas.iterrows() ): 
    structure = row1['Structure']
    #fig = plt.figure()
    fig, ax = plt.subplots(figsize=(6, 4))
    #plt.rcParams["figure.figsize"] = [10, 6]

    x1 = row1['X']
    y1 = row1['Y']
    z1 = row1['Z']
    x2 = row2['X']
    y2 = row2['Y']
    z2 = row2['Z']
    xdistance = abs(x1 - x2)
    ydistance = abs(y1 - y2)
    zdistance = abs(z1 - z2)
    distance = calculate_distance(np.array([x1,y1,z1]), np.array([x2,y2,z2]))

    counts = [xdistance, ydistance, zdistance, distance]
    bar_labels = ['X error', 'Y error', 'Z error', 'Distance error']
    bar_colors = ['tab:red', 'tab:blue', 'tab:green', 'tab:orange']
    #ax.bar(errors, counts, label=bar_labels, color=bar_colors)
    ax.set_ylabel('Error in um')
    # Label with specially formatted floats
    rects1 = ax.bar(x - width/2, counts, label=bar_labels, color=bar_colors)
    ax.bar_label(rects1, padding=2)
    ax.set_xticks(x, errors)
    ax.set_title('Errors for ' + structure)
    ax.set_ylim([0,1200])
    #ax.legend(title='Error color')
    plt.close()
    outpath = os.path.join(distancepath, 'eps', f'{structure}_error.eps')
    fig.savefig(outpath, bbox_inches="tight", format='eps')
    outpath = os.path.join(distancepath, f'{structure}_error.png')
    fig.savefig(outpath, bbox_inches="tight")


In [ ]:
print(f'n={len(distances)}, min={min(distances)} max={max(distances)}, mean={np.mean(distances)}')
ds = {k: v for k, v in sorted(sortme.items(), key=lambda item: item[1])}
for structure, d in ds.items():
    
    #print(f'{structure} distance from Allen={round(d,2)} micrometers')
    print('<tr>')
    print(f'<td><img src="https://www.brainsharer.org/images/{structure}.png"></td>')
    print(f'<td><img src="https://www.brainsharer.org/images/{structure}_error.png"></td>')
    print('</tr>')


In [ ]:
df_midbrain_distance = pd.read_csv(os.path.join(csvpath, 'using_midbrain_distance.csv'))
df_midbrain_allen = pd.read_csv(os.path.join(csvpath, 'using_midbrain_allen.csv'))
df_midbrain_atlas = pd.read_csv(os.path.join(csvpath, 'using_midbrain_atlas.csv'))

In [ ]:
df_all_distance = pd.read_csv(os.path.join(csvpath, 'using_all_distance.csv'))
df_all_allen = pd.read_csv(os.path.join(csvpath, 'using_all_allen.csv'))
df_all_atlas = pd.read_csv(os.path.join(csvpath, 'using_all_atlas.csv'))

In [ ]:
df_brainstem_distance = pd.read_csv(os.path.join(csvpath, 'using_brainstem_distance.csv'))
df_brainstem_allen = pd.read_csv(os.path.join(csvpath, 'using_brainstem_allen.csv'))
df_brainstem_atlas = pd.read_csv(os.path.join(csvpath, 'using_brainstem_atlas.csv'))

In [ ]:
midbrain_distances = df_midbrain_distance.distance
all_distances = df_all_distance.distance
brainstem_distances = df_brainstem_distance.distance
fig, ax = plt.subplots(figsize=(8, 6))
columns = [midbrain_distances, all_distances, brainstem_distances]
ax.boxplot(columns, patch_artist=True)
ax.set_title("Distances between the DK atlas and the Allen (um)")
title1 = f"Using mid-brain\nn={len(midbrain_distances)}, min={round(min(midbrain_distances))}"
title1 += f"\nmax={round(max(midbrain_distances))}, mean={round(np.mean(midbrain_distances))}"
title2 = f"Using all structures\nn={len(all_distances)}, min={round(min(all_distances))}"
title2 += f"\nmax={round(max(all_distances))}, mean={round(np.mean(all_distances))}"
title3 = f"Using brainstem structures\nn={len(brainstem_distances)}, min={round(min(brainstem_distances))}"
title3 += f"\nmax={round(max(brainstem_distances))}, mean={round(np.mean(brainstem_distances))}"
plt.xticks([1,2,3], [title1, title2, title3])
plt.show()

In [ ]:
len(all_distances), min(all_distances), max(all_distances), np.mean(all_distances)

In [ ]:
for i, row in df_midbrain_distance.iterrows():
    print(f'{row[0]} distance={round(row[1],2)}')

In [ ]:
for i, row in df_brainstem_distance.iterrows():
    print(f'<tr><td>{row[0]}</td> <td>{round(row[1],2)}</td></tr>')

In [ ]:
xmin = 8000
xmax = 13000
ymin = 2000
ymax = 6000
zmin = 0
zmax = 25*456

In [ ]:
df_midbrain_atlas.head(2)

In [ ]:
for color, label in zip('bg', ['midbrain', 'brainstem']):
    allen_subset = df_midbrain_allen[df_midbrain_allen.area == label]
    atlas_subset = df_midbrain_atlas[df_midbrain_atlas.area == label]
    plt.scatter(allen_subset.X, allen_subset.Y, s=120, marker='^', c=color, label=str('Allen ' + label))
    plt.scatter(atlas_subset.X, atlas_subset.Y, s=120, marker='o', c=color, label=str('Atlas ' + label))
    plt.xlabel('X')
    plt.ylabel('Y')
    ax = plt.gca()
    ax.set_xlim([xmin, xmax])
    ax.set_ylim([ymin, ymax])
    ax.invert_yaxis()
    #ax.annotate('x', (atlas_subset.X, atlas_subset.Y))
    #ax.annotate('x', xy=(atlas_subset.X, atlas_subset.Y))
plt.title('Mid-brain Allen/Altas X Y scatter plot')    
plt.legend()
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)
for k, v in df_midbrain_atlas.iterrows():
    ax.annotate(v['Structure'],
                xy=(v['X'], v['Y']),
                xytext=(10, 10), textcoords='offset points',
                family='sans-serif', fontsize=10, color='black')

In [ ]:
for color, label in zip('bgr', ['midbrain', 'brainstem']):
    allen_subset = df_midbrain_allen[df_midbrain_allen.area == label]
    atlas_subset = df_midbrain_atlas[df_midbrain_atlas.area == label]
    plt.scatter(allen_subset.X, allen_subset.Z, s=120, marker='^', c=color, label=str('Allen ' + label))
    plt.scatter(atlas_subset.X, atlas_subset.Z, s=120, marker='o', c=color, label=str('Atlas ' + label))
    plt.xlabel('X')
    plt.ylabel('Z')
    ax = plt.gca()
    ax.set_xlim([xmin, xmax])
    #ax.set_ylim([ymin, ymax])
    #ax.invert_yaxis()
plt.title('Mid-brain Allen/Altas X Z scatter plot')    
plt.legend()
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)

In [ ]:
for color, label in zip('bgr', ['midbrain', 'brainstem']):
    allen_subset = df_midbrain_allen[df_midbrain_allen.area == label]
    atlas_subset = df_midbrain_atlas[df_midbrain_atlas.area == label]
    plt.scatter(allen_subset.Z, allen_subset.Y, s=120, marker='^', c=color, label=str('Allen ' + label))
    plt.scatter(atlas_subset.Z, atlas_subset.Y, s=120, marker='o', c=color, label=str('Atlas ' + label))
    plt.xlabel('Z')
    plt.ylabel('Y')
    ax = plt.gca()
    #ax.set_xlim([xmin, xmax])
    #ax.set_ylim([ymin, ymax])
    ax.invert_yaxis()
plt.title('Midbrain Allen/Altas Y Z scatter plot')    
plt.legend()
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)

In [ ]:
for color, label in zip('bgr', ['midbrain', 'brainstem']):
    allen_subset = df_all_allen[df_all_allen.area == label]
    atlas_subset = df_all_atlas[df_all_atlas.area == label]
    plt.scatter(allen_subset.X, allen_subset.Y, s=120, marker='^', c=color, label=str('Allen ' + label))
    plt.scatter(atlas_subset.X, atlas_subset.Y, s=120, marker='o', c=color, label=str('Atlas ' + label))
    plt.xlabel('X')
    plt.ylabel('Y')
    ax = plt.gca()
    ax.set_xlim([xmin, xmax])
    ax.set_ylim([ymin, ymax])
    ax.invert_yaxis()
plt.title('All structures - Allen/Altas X Y scatter plot')    
plt.legend()
plt.legend()
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)

In [ ]:
for color, label in zip('bgr', ['midbrain', 'brainstem']):
    allen_subset = df_all_allen[df_all_allen.area == label]
    atlas_subset = df_all_atlas[df_all_atlas.area == label]
    plt.scatter(allen_subset.X, allen_subset.Z, s=120, marker='^', c=color, label=str('Allen ' + label))
    plt.scatter(atlas_subset.X, atlas_subset.Z, s=120, marker='o', c=color, label=str('Atlas ' + label))
    plt.xlabel('X')
    plt.ylabel('Z')
    ax = plt.gca()
    ax.set_xlim([xmin, xmax])
    #ax.set_ylim([ymin, ymax])
    #ax.invert_yaxis()
plt.title('All structures - Allen/Altas X Z scatter plot')    
plt.legend()
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)

In [ ]:
for color, label in zip('bgr', ['midbrain', 'brainstem']):
    allen_subset = df_all_allen[df_all_allen.area == label]
    atlas_subset = df_all_atlas[df_all_atlas.area == label]
    plt.scatter(allen_subset.Z, allen_subset.Y, s=120, marker='^', c=color, label=str('Allen ' + label))
    plt.scatter(atlas_subset.Z, atlas_subset.Y, s=120, marker='o', c=color, label=str('Atlas ' + label))
    plt.xlabel('Z')
    plt.ylabel('Y')
    ax = plt.gca()
    #ax.set_xlim([xmin, xmax])
    #ax.set_ylim([ymin, ymax])
    ax.invert_yaxis()
plt.title('All structures - Allen/Altas Y Z scatter plot')    
plt.legend()
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)

In [ ]:
# xy
for color, label in zip('bgr', ['Left', 'Right', 'Singular']):
    allen_subset = df_brainstem_allen[df_brainstem_allen.S == label]
    atlas_subset = df_brainstem_atlas[df_brainstem_atlas.S == label]
    plt.scatter(allen_subset.X, allen_subset.Y, s=120, marker='^', c=color, label=str('Allen ' + label))
    plt.scatter(atlas_subset.X, atlas_subset.Y, s=120, marker='o', c=color, label=str('Atlas ' + label))
    plt.xlabel('X')
    plt.ylabel('Y')
    ax = plt.gca()
    ax.set_xlim([xmin, xmax])
    ax.set_ylim([ymin, ymax])
    ax.invert_yaxis()
plt.title('Brainstem Allen/Altas X Y scatter plot')    
plt.legend()
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)

In [ ]:
# xz
for color, label in zip('bgr', ['Left', 'Right', 'Singular']):
    allen_subset = df_brainstem_allen[df_brainstem_allen.S == label]
    atlas_subset = df_brainstem_atlas[df_brainstem_atlas.S == label]
    plt.scatter(allen_subset.X, allen_subset.Z, s=120, marker='^', c=color, label=str('Allen ' + label))
    plt.scatter(atlas_subset.X, atlas_subset.Z, s=120, marker='o', c=color, label=str('Atlas ' + label))
    plt.xlabel('X')
    plt.ylabel('Z')
    ax = plt.gca()
    ax.set_xlim([xmin, xmax])
    #ax.set_ylim([ymin, ymax])
    #ax.invert_yaxis()
plt.title('Brainstem Allen/Altas X Z scatter plot')    
plt.legend()
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)

In [ ]:
# yz
for color, label in zip('bgr', ['Left', 'Right', 'Singular']):
    allen_subset = df_brainstem_allen[df_brainstem_allen.S == label]
    atlas_subset = df_brainstem_atlas[df_brainstem_atlas.S == label]
    plt.scatter(allen_subset.Z, allen_subset.Y, s=120, marker='^', c=color, label=str('Allen ' + label))
    plt.scatter(atlas_subset.Z, atlas_subset.Y, s=120, marker='o', c=color, label=str('Atlas ' + label))
    plt.xlabel('Z')
    plt.ylabel('Y')
    ax = plt.gca()
    #ax.set_xlim([xmin, xmax])
    #ax.set_ylim([ymin, ymax])
    ax.invert_yaxis()
plt.title('Brainstem Allen/Altas Y Z scatter plot')    
plt.legend()
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)